In [1]:
from datasets import load_dataset

ds = load_dataset("cirimus/super-emotion")

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.44M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412059 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/51443 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/56310 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [4]:
tokenized_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/412059 [00:00<?, ? examples/s]

Map:   0%|          | 0/51443 [00:00<?, ? examples/s]

Map:   0%|          | 0/56310 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [7]:
labels = ["neutral", "surprise", "fear", "sadness", "joy", "anger", "love"]
id2label = dict(zip(range(len(labels)), labels))
label2id = dict(zip(labels, range(len(labels))))

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir="super-detection",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="super-emotion")
classifier(text)